In [2]:
import numpy as np
import pandas as pd
from collections import OrderedDict, Counter
import itertools
import plotly.express as px

In [3]:
meital = pd.read_csv('../csvs/meital.csv', parse_dates=['StartDate', 'StartTime'])

In [4]:
date_time = pd.to_datetime(meital['StartDate'].astype('string') + " " + meital['StartTime'].astype('string'))
meital = meital.drop(columns=['StartDate', 'StartTime'])
group_all_mices = meital.set_index(date_time)
group_all_mices

,Cage,In_g
2020-10-01 15:17:26-18:00,1,0.004
2020-10-01 16:11:42-18:00,1,0.084
2020-10-01 16:44:55-18:00,1,0.126
2020-10-01 17:31:15-18:00,1,0.020
2020-10-01 17:43:18-18:00,1,0.005
...,...,...
2020-10-18 05:04:43-18:00,16,0.292
2020-10-18 05:16:39-18:00,16,0.461
2020-10-18 06:56:53-18:00,16,0.008
2020-10-18 07:22:16-18:00,16,0.254


In [13]:
index_datetime = group_all_mices.index
index_datetime

DatetimeIndex(['2020-10-01 15:17:26-18:00', '2020-10-01 16:11:42-18:00',
               '2020-10-01 16:44:55-18:00', '2020-10-01 17:31:15-18:00',
               '2020-10-01 17:43:18-18:00', '2020-10-01 17:55:10-18:00',
               '2020-10-01 18:15:15-18:00', '2020-10-01 18:52:56-18:00',
               '2020-10-01 19:11:13-18:00', '2020-10-01 19:39:23-18:00',
               ...
               '2020-10-17 20:01:04-18:00', '2020-10-17 21:40:22-18:00',
               '2020-10-17 23:00:43-18:00', '2020-10-18 01:08:42-18:00',
               '2020-10-18 03:08:13-18:00', '2020-10-18 05:04:43-18:00',
               '2020-10-18 05:16:39-18:00', '2020-10-18 06:56:53-18:00',
               '2020-10-18 07:22:16-18:00', '2020-10-18 07:43:28-18:00'],
              dtype='datetime64[ns, pytz.FixedOffset(-1080)]', length=5125, freq=None)

In [9]:
def count_custome_day_start_and_end(pd_index_datetime, shift_hour):
    dates = sorted(set(pd_index_datetime.date))
    conditiones = [ logic 
        for date in dates 
        for logic in [
          (np.logical_and((pd_index_datetime.hour < shift_hour), (pd_index_datetime.date == date))), # day before
          (np.logical_and((pd_index_datetime.hour >= shift_hour), (pd_index_datetime.date == date)))] # day after
      ]
    langth = (len(conditiones)//2) + 1
    choices = [0, *np.repeat(range(1, langth), 2)]
    choices.pop()
    return np.select(conditiones, choices, pd_index_datetime.factorize()[0])
group_all_mices['days_exp'] = count_custome_day_start_and_end(index_datetime, 8)
group_all_mices

,Cage,In_g,days_exp
2020-10-01 15:17:26-18:00,1,0.004,1
2020-10-01 16:11:42-18:00,1,0.084,1
2020-10-01 16:44:55-18:00,1,0.126,1
2020-10-01 17:31:15-18:00,1,0.020,1
2020-10-01 17:43:18-18:00,1,0.005,1
...,...,...,...
2020-10-18 05:04:43-18:00,16,0.292,17
2020-10-18 05:16:39-18:00,16,0.461,17
2020-10-18 06:56:53-18:00,16,0.008,17
2020-10-18 07:22:16-18:00,16,0.254,17


In [11]:
sums_exp_days = group_all_mices.groupby(['days_exp', 'Cage']).sum()
sums_exp_days

In_g
days_exp Cage       
1        1     2.000
         2     2.773
         3     3.534
         4     2.919
         5     3.328
...              ...
18       1     0.004
         2     0.112
         6     0.020
         7     0.004
         9     0.091

[275 rows x 1 columns]

In [12]:
px.box(x=sums_exp_days.index.get_level_values(0), y=sums_exp_days['In_g'], template='simple_white')